# 0. Install and Import dependencies

In [ ]:
#!pip install tensorflow-gpu==1.15.0 tensorflow==1.15.0 stable-baselines gym-anytrading-debug gym

In [ ]:
# Gym stuff
import gym
import gym_anytrading_debug

# Stable baselines - rl stuff
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C

# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# 1. Bring in Crypto ETH/USDT Data 
https://www.cryptodatadownload.com/data/binance/

In [ ]:
df = pd.read_csv('data/Binance_ETHUSDT_minute.csv')
df.head(15)

In [ ]:
df.dtypes

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.drop(columns=['unix','symbol','Volume ETH'])
df.rename(columns={'Volume USDT': 'volume'}, inplace=True)

In [ ]:
df.sort_values('date', ascending=True, inplace=True)
df.set_index('date', inplace=True)
df.head()

In [ ]:
df.dtypes

# 2. Build Environment

In [ ]:
nb_data_rows = len(df.index)
print(nb_data_rows)
window_size = 120
env = gym.make('mystocks-v0', df=df, frame_bound=(window_size, 200000), window_size=window_size, debug=True)

In [ ]:
#env.signal_features

In [ ]:
env.action_space

In [ ]:
#state = env.reset()
#while True: 
    #action = env.action_space.sample()
    #n_state, reward, done, info = env.step(action)
    #if done: 
        #print("info", info)
        #break
        
#plt.figure(figsize=(15,6))
#plt.cla()
#env.render_all()
#plt.show()

# 2.1 Add Custom Indicators

## 2.1.1. Install and Import New Dependencies

In [ ]:
#!pip install finta

In [ ]:
from gym_anytrading_debug.envs import MyStocksEnv
from finta import TA

## 2.1.3. Calculate SMA, RSI and OBV

In [ ]:
#df['SMA'] = TA.SMA(df, 12)
#df['RSI'] = TA.RSI(df)
#df['OBV'] = TA.OBV(df)
#df.fillna(0, inplace=True)

In [ ]:
t = TA.MACD(df)
#t.head(15)

df['MACD'] = t['MACD']
df['SIGNAL'] = t['SIGNAL']
df.fillna(0, inplace=True)
df.head()

In [ ]:
df.head(15)

## 2.1.4. Create New Environments

In [ ]:
def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'low'].to_numpy()[start:end]
    #signal_features = env.df.loc[:, ['low','volume','tradecount','SMA', 'RSI', 'OBV']].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['low','volume','tradecount','MACD', 'SIGNAL']].to_numpy()[start:end]
    return prices, signal_features

In [ ]:
class MyCustomEnv(MyStocksEnv):
    init_data = add_signals
    
env2 = MyCustomEnv(df=df, window_size=window_size, frame_bound=(100000,200000), debug=True)

In [ ]:
#env2.signal_features

# 3. Build Environment and Train

In [ ]:
env_maker = lambda: env2
env = DummyVecEnv([env_maker])

In [ ]:
model = A2C('MlpLstmPolicy', env, verbose=1) 
model.learn(total_timesteps=1000000)

In [ ]:
import os
ppo_path = os.path.join('SavedModels', 'A2C_Trading_model_winsize120')
model.save(ppo_path)

# 4. Evaluation

In [ ]:
env = MyCustomEnv(df=df, window_size=window_size, frame_bound=(100000,200000), debug=True)
obs = env.reset()
while True: 
    obs = obs[np.newaxis, ...]
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        print("info", info)
        break

In [ ]:
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()